In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/100_Unique_QA_Dataset.csv')

df.shape

(90, 2)

In [3]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


**1. Tokenize words**

In [4]:
def tokenize(text):
    text = text.lower()
    text = text.replace('?', '')
    text = text.replace("'", "")
    return text.split()

In [5]:
tokenize("What is the capital of france?")

['what', 'is', 'the', 'capital', 'of', 'france']

**2. Create Vocabulary**

In [6]:
vocab = {'<UNK>':0}

In [7]:
def build_vocab(row):
    tokenized_question = tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])
    
    tokens = tokenized_question + tokenized_answer
    
    for token in tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

In [8]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [9]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [10]:
print("Vocabulary length:", len(vocab) - 1)

Vocabulary length: 323


**3. Convert words to numeric indices**

In [11]:
def text_to_indices(text, vocab):
    indexed_text = []
    
    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])            
    
    return indexed_text        
        

In [12]:
text_to_indices("What is the capital of France?", vocab)

[1, 2, 3, 4, 5, 6]

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

In [14]:
class CustomDataset(Dataset):
    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
            question = self.df.iloc[index]['question']
            answer = self.df.iloc[index]['answer']
            
            numeric_question = text_to_indices(question, self.vocab)
            numeric_answer = text_to_indices(answer, self.vocab)
            
            return torch.tensor(numeric_question), torch.tensor(numeric_answer)

In [15]:
dataset = CustomDataset(df, vocab)

In [16]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [17]:
for question, answer in dataloader:
    print(question, answer)

tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([[121]])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([[273]])
tensor([[ 42, 137,   2, 226,  12,   3, 227, 228]]) tensor([[155]])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([[16]])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([[225]])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([[321]])
tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([[85]])
tensor([[ 1,  2,  3, 69,  5,  3, 70, 71]]) tensor([[72]])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([[166]])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([[68]])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([[136]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[  1,   2,   3, 122, 123,  19,   3,  45]]) tensor([[124]])
tensor([[10, 96,  3, 97]]) tensor([[98]])
tensor([[ 42,  86,  87, 241, 242,  19,  39, 243]]) tensor([[244]])
tensor([[ 10, 140,   3, 141

**Architecture**

50-neuron input embeddings -> 64-neuron hidden layer -> 324-neuron output layer (multi-classification) 

In [18]:
import torch.nn as nn

In [51]:
class SimpleRNN(nn.Module):
    def __init__(self, vocab_len):
        super().__init__()
        
        # Architecture
        self.embedding = nn.Embedding(vocab_len, embedding_dim=50)
        self.rnn = nn.RNN(50, 64, batch_first=True) # outputs hidden state outputs + final output
        self.fc = nn.Linear(64, vocab_len)
        
    def forward(self, question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final).squeeze(0)
        
        return output

In [46]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[9][0]
print(f"shape of a: {a.shape}")

b = x(a)
print(f"shape of b: {b.shape}")

_, c = y(b)
print(f"shape of c: {c.shape}")

d = z(c)
print(f"shape of d: {d.shape}")

shape of a: torch.Size([8])
shape of b: torch.Size([8, 50])
shape of c: torch.Size([1, 64])
shape of d: torch.Size([1, 324])


In [58]:
learning_rate = 0.001
epochs = 50

In [59]:
model = SimpleRNN(len(vocab))

In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [61]:
for epoch in range(epochs):
    total_loss = 0
    for question, answer in dataloader:
        optimizer.zero_grad()
        
        output = model(question)
        
        loss = criterion(output, answer[0])
        
        loss.backward()
        
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch [{epoch}/{epochs}], Loss: {total_loss:4f}")        

Epoch [0/50], Loss: 527.216840
Epoch [1/50], Loss: 452.534630
Epoch [2/50], Loss: 372.811245
Epoch [3/50], Loss: 314.163065
Epoch [4/50], Loss: 264.385934
Epoch [5/50], Loss: 216.772181
Epoch [6/50], Loss: 173.758455
Epoch [7/50], Loss: 135.951317
Epoch [8/50], Loss: 105.171187
Epoch [9/50], Loss: 81.046510
Epoch [10/50], Loss: 62.869826
Epoch [11/50], Loss: 48.985862
Epoch [12/50], Loss: 39.167432
Epoch [13/50], Loss: 31.908893
Epoch [14/50], Loss: 25.996769
Epoch [15/50], Loss: 21.678940
Epoch [16/50], Loss: 18.240517
Epoch [17/50], Loss: 15.634372
Epoch [18/50], Loss: 13.301234
Epoch [19/50], Loss: 11.531136
Epoch [20/50], Loss: 10.106783
Epoch [21/50], Loss: 8.794779
Epoch [22/50], Loss: 7.779031
Epoch [23/50], Loss: 6.925852
Epoch [24/50], Loss: 6.175720
Epoch [25/50], Loss: 5.581178
Epoch [26/50], Loss: 5.048021
Epoch [27/50], Loss: 4.574760
Epoch [28/50], Loss: 4.168552
Epoch [29/50], Loss: 3.805860
Epoch [30/50], Loss: 3.493363
Epoch [31/50], Loss: 3.227553
Epoch [32/50], Loss:

In [81]:
def predict(model, question, threshold=0.5):
    # Convert question to indices
    numeric_question = text_to_indices(question, vocab)
    
    question_tensor = torch.tensor(numeric_question).unsqueeze(0)
    
    output = model(question_tensor)
    
    probabilities = torch.nn.functional.softmax(output, dim=1)
    
    value, index = torch.max(probabilities, dim=1)
    
    print("Confidence: ", value)
    
    if value < threshold:
        print("I don't know")
        return
    
    print(list(vocab.keys())[index])              
    

In [84]:
predict(model, "what is a france")

Confidence:  tensor([0.9479], grad_fn=<MaxBackward0>)
paris
